<a href="https://colab.research.google.com/github/ExploitIdF/Eclairage/blob/main/rappor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd, numpy as np,re,os,glob,io
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
pd.set_option('display.max_colwidth', 150)
os.environ["GOOGLE_CLOUD_PROJECT"] = "batiment-dirif"
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from google.colab import auth
auth.authenticate_user()
if not firebase_admin._apps:
    cred = credentials.ApplicationDefault()
    firebase_admin.initialize_app(cred)


# Configuration


In [2]:
issues=pd.read_csv('https://raw.githubusercontent.com/ExploitIdF/Referentiel_Tunnels/refs/heads/main/issuesFermetures.csv')
niches=pd.read_csv('https://raw.githubusercontent.com/ExploitIdF/Referentiel_Tunnels/refs/heads/main/nichesFermetures.csv')

In [3]:
niches[:1]

,CodeEx,Tatouage,triCode,Sens,Fermeture
0,DEF-NSW-701/DEF,L10.122A,A14,W,A14&NEU-W


In [4]:
issues[:1]

,CodeEx,PC,Tatouage,triCode,Sens,Fermeture
0,IS101,PCO,L10.366R,A14,Y,A14&NEU-Y


### Fermetures


In [52]:
nbrNi=niches.groupby('Fermeture')['tv'].count().rename('nbrNi')
niches['tv']=niches['Tatouage']+','
nicFer=niches.groupby('Fermeture')['tv'].sum()
nicFer=nicFer.apply(lambda x: '['+x[:-1]+']').rename('niches')

nbrIs=issues.groupby('Fermeture')['tv'].count().rename('nbrIs')
issues['tv']=issues['Tatouage']+','
issFer=issues.groupby('Fermeture')['tv'].sum()
issFer=issFer.apply(lambda x: '['+x[:-1]+']').rename('issues')

fermetures=pd.concat([nicFer,issFer,nbrIs,nbrNi],axis=1).reset_index().rename(columns={'Fermeture': 'nom'})
fermetures[:1]


,nom,niches,issues,nbrIs,nbrNi
0,A14&NEU-W,"[L10.122A,L10.123C,L10.124D,L10.125E,L10.126F,L10.159U,L10.161W,L10.162X,L10.163Y,L10.164A,L10.165C,L10.166D,L10.167E,L10.168F,L10.169G,L10.171J,L...","[L10.396C,L10.397D,L10.398E,L10.399F,L10.401H,L10.402J,L10.403K,L10.404L,L10.405M,L10.406N,L10.407P,L10.408R,L10.409S,L10.411U,L10.412V,L10.413W,L...",36,46


In [39]:
if False:
  db = firestore.client()
  collection_name = 'fermetures'
  for index, row in fermetures.iloc[:,:-2].iterrows():
      try:
          doc_data = row.to_dict()
          db.collection(collection_name).add(doc_data)
      except Exception as e:
          print(f"Error adding document at index {index}: {e}")


### config_app  pour les PC
```
  codePC: string;
  descPC: string;
  codeRC: string;
  descRC: string;    
  commentaireObligatoire: boolean;
  niveau: string;
```


In [37]:
config_app=pd.read_csv('https://storage.googleapis.com/kzxmvqybhjsdpfgwcnrualteioxcfm/controlesIsNi.csv').iloc[:,:-3]
#['codePC', 'PC', 'codeRC', 'RC', 'Probab', 'Issue_VB', 'Issue_IA', 'Niche_VB', 'Niche_IA', 'CME compromis','Anomalie résolué durant la prestation','Commentaire pour décrire l'anomalie constatée']

niveaux=['IS_VS','IS_VD','IS_MA','NI_VD','NI_MA']
pcNiv=config_app.groupby(['codePC'])[['Issue_VB', 'Issue_IA', 'Niche_VB', 'Niche_IA']].first()
pcNiv.loc[pcNiv['Issue_IA']=='Oui', 'niveau']='IS_MA'
pcNiv.loc[pcNiv['Issue_VB']=='Oui', 'niveau']='IS_VB'
pcNiv.loc[pcNiv['Niche_IA']=='Oui', 'niveau']='NI_MA'
pcNiv.loc[pcNiv['Niche_VB']=='Oui', 'niveau']='NI_VB'
config_app=config_app.iloc[:,:-5].join(pcNiv['niveau'],on='codePC')
config_app['commentaireObligatoire']='true'
config_app.loc[config_app['RC'].str[0]=='2' ,'commentaireObligatoire']='false'
config_app[:9]
config_app.columns=['codePC', 'descPC', 'codeRC', 'descRC', 'niveau', 'commentaireObligatoire']


In [38]:
collection_name = 'point_controle_conf'
for index, row in config_app.iterrows():
    try:
        doc_data = row.to_dict()
        db.collection(collection_name).add(doc_data)
    except Exception as e:
        print(f"Error adding document at index {index}: {e}")



# Création des OT
Il faut créer deux tables : 'OT_commande' & 'OT_statut'

```
  id: string;
  codeOT: string;
  codeIssueNiche: string;
  niveau:  string;
  fermetureId: string;
  date_creation: Date;
  date_prevue: Date;

OT_statut
  id: string;
  codeOT: string;
  statut: 'commande'| 'en_cours' | 'termine' ;
```


In [58]:
import random
tmpF=fermetures.copy().reset_index()
tmpF['date_prevue']=[random.choice(['2025-09-15','2025-09-16','2025-09-25','2025-09-26','2025-10-15','2025-10-16']) for n in range(len(tmpF))]
tmpF['codeOT']=(tmpF['index']+10).astype(str)
tmpF[:1]

,index,nom,niches,issues,nbrIs,nbrNi,date_prevue,codeOT
0,0,A14&NEU-W,"[L10.122A,L10.123C,L10.124D,L10.125E,L10.126F,L10.159U,L10.161W,L10.162X,L10.163Y,L10.164A,L10.165C,L10.166D,L10.167E,L10.168F,L10.169G,L10.171J,L...","[L10.396C,L10.397D,L10.398E,L10.399F,L10.401H,L10.402J,L10.403K,L10.404L,L10.405M,L10.406N,L10.407P,L10.408R,L10.409S,L10.411U,L10.412V,L10.413W,L...",36,46,2025-10-15,10


In [76]:
lstA=[]
for niveau in ['IS_VD','IS_MA']:
  for k in range(len(tmpF)):
    lf=(tmpF.loc[k,'nbrIs'])
    for i in range(lf):
      codeOT='976'+tmpF.loc[k,'codeOT']+str(110+i)
      codeIssueNiche=tmpF.loc[k,'issues'][1:-1].split(',')[i]
      lstA.append({'codeOT':codeOT,'codeIssueNiche':codeIssueNiche,'niveau': niveau,'fermetureId':tmpF.loc[k,'nom'],'date_creation':'2025-07-28','date_prevue':tmpF.loc[k,'date_prevue']})

lstB=[]
for niveau in ['NI_VD','NI_MA']:
  for k in range(len(tmpF)):
    lf=(tmpF.loc[k,'nbrNi'])
    for i in range(lf):
      codeOT='976'+tmpF.loc[k,'codeOT']+str(210+i)
      codeIssueNiche=tmpF.loc[k,'niches'][1:-1].split(',')[i]
      lstB.append({'codeOT':codeOT,'codeIssueNiche':codeIssueNiche,'niveau': niveau,'fermetureId':tmpF.loc[k,'nom'],'date_creation':'2025-07-28','date_prevue':tmpF.loc[k,'date_prevue']})

ot_commande=pd.DataFrame(lstA+lstB)
ot_statut=ot_commande[['codeOT', 'fermetureId']].copy()
ot_statut['statut']='commande'

In [72]:
if False :
  collection_name = 'ot_commande'
  for index, row in ot_commande.iterrows():
      try:
          doc_data = row.to_dict()
          db.collection(collection_name).add(doc_data)
      except Exception as e:
          print(f"Error adding document at index {index}: {e}")

  collection_name = 'ot_statut'
  for index, row in ot_statut.iterrows():
      try:
          doc_data = row.to_dict()
          db.collection(collection_name).add(doc_data)
      except Exception as e:
          print(f"Error adding document at index {index}: {e}")

In [78]:
collection_name = 'ot_statut'
for index, row in ot_statut.iterrows():
    try:
        doc_data = row.to_dict()
        db.collection(collection_name).add(doc_data)
    except Exception as e:
        print(f"Error adding document at index {index}: {e}")

In [77]:
ot_statut[:9]

,codeOT,fermetureId,statut
0,97610110,A14&NEU-W,commande
1,97610111,A14&NEU-W,commande
2,97610112,A14&NEU-W,commande
3,97610113,A14&NEU-W,commande
4,97610114,A14&NEU-W,commande
5,97610115,A14&NEU-W,commande
6,97610116,A14&NEU-W,commande
7,97610117,A14&NEU-W,commande
8,97610118,A14&NEU-W,commande
